<div align="center">
  <h1>Welcom to SSCMA for Google Colab Training Example 🔥 </h1>
  <a href="https://sensecraftma.seeed.cc/" target="_blank"><img width="20%" src="https://files.seeedstudio.com/sscma/docs/images/SSCMA-Hero.png"></a>
</div>

## ⚠️ Important Notice

Before you begin, please ensure:

1. **Enable GPU Acceleration**: Click `Runtime` → `Change runtime type` → Select `GPU` in `Hardware accelerator`
2. **Colab Version Requirement**: **You MUST use Colab version 2025.07** for compatibility. Other versions may cause unexpected errors.

If you encounter version-related issues, you can run the following commands in the first code cell to check your environment:

```python
# Check if GPU is available
!nvidia-smi

# Check Python version
!python --version
```

# person Detection - Swift-YOLO

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seeed-studio/sscma-model-zoo/blob/main/notebooks/en/person_Detection_Swift-YOLO_192.ipynb)

**Version:** 1.0.0

**Category:** Object Detection

**Algorithm:** [Swift-YOLO](configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py)

**Dataset:** [Person](https://universe.roboflow.com/hanzhou-7mktt/ssperson/dataset/7#)

**⚠️ IMPORTANT: To download the dataset, you need to set your Roboflow API key.**

1. Go to your [Roboflow Settings](https://app.roboflow.com/settings/api).
2. Copy your private API key.
3. In Colab, click the **key icon** on the **left sidebar** to open Secrets.
4. Add a new secret named `ROBOFLOW_API_KEY` with your API key as the value.

**Class:** `person`

![person Detection](https://files.seeedstudio.com/sscma/static/detection_person.png)

The model is a Swift-YOLO model trained on the person detection dataset.



## ⚙️Prerequisites
### Setup SSCMA
Clone the [repository](https://github.com/Seeed-Studio/ModelAssistant) and install the dependencies.

In [1]:
!pip install ethos-u-vela
!git clone https://github.com/Seeed-Studio/ModelAssistant.git -b 2.0.0  #clone the repo
%cd ModelAssistant
!. ./scripts/setup_colab.sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.12.19
    Uninstalling flatbuffers-25.12.19:
      Successfully uninstalled flatbuffers-25.12.19
Cloning into 'ModelAssistant'...
remote: Enumerating objects: 15755, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 15755 (delta 7), reused 17 (delta 2), pack-reused 15719 (from 1)
Receiving objects: 100% (15755/15755), 26.60 MiB | 22.76 MiB/s, done.
Resolving deltas: 100% (9193/9193), done.
/content/ModelAssistant
Checking if CUDA available... Not found!
Please enable GPU Runtime


### Download the pretrain model weights file

In [ ]:
%mkdir -p person_Detection_Swift-YOLO_192
!wget -c https://files.seeedstudio.com/sscma/model_zoo/detection/person/person_detection.pth -O person_Detection_Swift-YOLO_192/pretrain.pth

### Download the dataset

In [ ]:
%mkdir -p person_Detection_Swift-YOLO_192
!pip install roboflow
import os
from google.colab import userdata
os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")
from roboflow import download_dataset
dataset = download_dataset("https://universe.roboflow.com/hanzhou-7mktt/ssperson/dataset/7", "coco")
dataset_path = dataset.location
import shutil
shutil.move(dataset_path, "person_Detection_Swift-YOLO_192/dataset")


## 🚀Train a model with SSCMA
All the training parameters are in the `config.py` file, you can change the parameters to train your own model.

Below are explanations of some common parameters. You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/config) for more details.
- `data_root` - the datasets path.
- `epochs`- the train epochs. **we use 10 epochs as an example**.
- `batch_size` - the batch size.
- `height` - the image height.
- `width` - the image width.
- `load_from` - the pretrained model path.
- `num_classes` - the number of classes.

You can overwrite the parameters in the `config.py` file by using the `--cfg-options` argument.
```bash
# Example
sscma.train config.py --cfg-options data_root=./datasets/test_dataset epochs=10
```

In [ ]:
!sscma.train configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
--cfg-options  \
    work_dir=person_Detection_Swift-YOLO_192 \
    num_classes=1 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=person_Detection_Swift-YOLO_192/dataset/ \
    load_from=person_Detection_Swift-YOLO_192/pretrain.pth

## 📦Export the model
After training, you can export the model to the format for deployment. SSCMA supports exporting to ONNX, and TensorFlow Lite at present.
You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/export/overview) for more details.

```bash
python3 tools/export.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>"
```

In [ ]:
import os
with open('person_Detection_Swift-YOLO_192/last_checkpoint', 'r') as f:
	os.environ['CHECKPOINT_FILE_PATH'] = f.read()

In [ ]:
!sscma.export configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py $CHECKPOINT_FILE_PATH --cfg-options  \
    work_dir=person_Detection_Swift-YOLO_192 \
    num_classes=1 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=person_Detection_Swift-YOLO_192/dataset/ \
    load_from=person_Detection_Swift-YOLO_192/pretrain.pth

### 📝Evaluate the model
After exporting the model, you can evaluate the model on the test dataset.
You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/export/overview) for more details.


```bash
python3 tools/inference.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>"
```

### Evaluate the PyTorch model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}.pth \
--cfg-options  \
    work_dir=person_Detection_Swift-YOLO_192 \
    num_classes=1 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=person_Detection_Swift-YOLO_192/dataset/ \
    load_from=person_Detection_Swift-YOLO_192/pretrain.pth

### Evaluate the ONNX model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.onnx \
--cfg-options  \
    work_dir=person_Detection_Swift-YOLO_192 \
    num_classes=1 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=person_Detection_Swift-YOLO_192/dataset/ \
    load_from=person_Detection_Swift-YOLO_192/pretrain.pth

### Evaluate the TFLite FLOAT32 model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.tflite \
--cfg-options  \
    work_dir=person_Detection_Swift-YOLO_192 \
    num_classes=1 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=person_Detection_Swift-YOLO_192/dataset/ \
    load_from=person_Detection_Swift-YOLO_192/pretrain.pth

### Evaluate the TFLite INT8 model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_int8.tflite \
--cfg-options  \
    work_dir=person_Detection_Swift-YOLO_192 \
    num_classes=1 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=person_Detection_Swift-YOLO_192/dataset/ \
    load_from=person_Detection_Swift-YOLO_192/pretrain.pth

## 🤖 Deploy the model
After model training, evaluation and export, you can deploy the model to your device. You can refer to [Documentation](https://sensecraftma.seeed.cc/deploy/overview) for more details.

In [ ]:
%ls -lh person_Detection_Swift-YOLO_192/

### Thanks for Trying Out SSCMA 🎉

Congratulations, you have completed this tutorial. If you are interested in more application scenarios or our projects, please feel free to give [SSCMA](https://github.com/Seeed-Studio/ModelAssistant) a star ✨ on GitHub.

If you have any questions about this tutorial, please also feel free to [submit an issue](https://github.com/Seeed-Studio/ModelAssistant/issues).